# Проект "Музыка больших городов"

На данных Яндекс Музыки необходимо сравнить поведение пользователей Москвы и Санкт-Петербурга

**Цель исследования** — проверка следующих утвреждений:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования** 
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


<a id='section_id'></a>
# Содержание



1. [Импорт библиотек и загрузка данных](#1)
2. [Предобработка данных](#2)
   - 2.1 [Переименование столбцов](#2.1)
   - 2.2 [Проверка пропусков](#2.2)
   - 2.3 [Поиск дубликатов](#2.3)
   - 2.4 [Вывод](#2.4)   
3. [EDA](#3)
   - 3.1 [Сравнение поведения пользователей двух столиц](#3.1)
   - 3.2 [Музыка в начале и в конце недели](#3.2)
   - 3.3 [Жанровые предпочтения в Москве и Петербурге](#3.3)
3. [Вывод](#4)  

<a id='1'></a>
## 1. Импорт библиотек и загрузка данных
[Назад к содержанию](#section_id)

Импортируем библиотеки, загружаем данные, а так же выводим информацию о датасете

In [1]:
import pandas as pd
df = pd.read_csv('...')
display(df.head())
print('Информация о датасете')
print('')
df.info()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


Информация о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — object.

Согласно документации к данным:

- `userID` — идентификатор пользователя;
- `Track` — название трека;
- `artist` — имя исполнителя;
- `genre` — название жанра;
- `City` — город пользователя;
- `time` — время начала прослушивания;
- `Day` — день недели.

- Имеются пропуски, а так же названия заголовков, написанные с ипользованием некорректного стиля.


## 2. Предобработка данных
<a id='2'></a>
[Назад к содержанию](#section_id)

### 2.1 Переименование столбцов
<a id='2.1'></a>


In [2]:
df = df.rename(columns = {'  userID':'user_id','Track':'track', '  City  ':'city','Day':'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Названия столбцов откорректированы

### 2.2 Проверка пропусков
<a id='2.2'></a>
[Назад к содержанию](#section_id)

In [3]:
display(pd.concat(
    [
        df.isna().sum(),
        df.isna().mean().apply('{:.4%}'.format)
    ], axis=1,
    keys=['nan_count', 'nan_share']
).sort_values(by='nan_count', ascending=False))

,nan_count,nan_share
artist,7203,11.0681%
track,1231,1.8915%
genre,1198,1.8408%
user_id,0,0.0000%
city,0,0.0000%
time,0,0.0000%
day,0,0.0000%


- Так как мы исследуем жанровые предпочтения, то пропуски в `artist` и `track` можно заполнить значением `unknown`.
- Пропуски в `genre` составляют менее 2% от общего числа, их можно удалить или так же заменить значением. описанным выше.

In [4]:
columns_to_replace = ['track','artist','genre']
for column in columns_to_replace:
    df[column]=df[column].fillna('unknown')# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [5]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Пропуски обработаны

### 2.3 Поиск дубликатов
<a id='2.3'></a>
[Назад к содержанию](#section_id)

In [6]:
print('Выявлено явных дубликатов:', df.duplicated().sum())

Выявлено явных дубликатов: 3826


In [7]:
# Удаляем явные дубликаты
df=df.drop_duplicates().reset_index(drop=True)
print('Выявлено явных дубликатов повторно:', df.duplicated().sum())

Выявлено явных дубликатов повторно: 0


In [8]:
# Поищем неявные дубликаты в жанрах
(df.genre.sort_values()).unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Выявлены следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

In [9]:
# Избавляемся от неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)

### 2.4 Вывод
<a id='2.4'></a>
[Назад к содержанию](#section_id)

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Стиль заголовков отредактирован, пропущенны значения обработаны заменой.
Явные дуликаты удалены, неявные обработы заменой



## 3. EDA
<a id='3'></a>
[Назад к содержанию](#section_id)

### 3.1 Сравнение поведения пользователей двух столиц
<a id='3.1'></a>

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга.
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [10]:
# Рассмотрим количество прослушиваний в разных городах
df.groupby('city')['user_id'].count().to_frame()

,user_id
city,
Moscow,42741
Saint-Petersburg,18512


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [11]:
# Рассмотрим количество прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count().to_frame()

,user_id
day,
Friday,21840
Monday,21354
Wednesday,18059


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Далее создаем функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города, используя два параметра:
* день недели,
* название города.


In [12]:
def number_tracks (day,city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Далее применим функцию и определим прослушивания в городах по дням.
Полученную информацию сформирую в таблицу для удобства восприятия.

In [13]:
pd.DataFrame(data=[['МСК',number_tracks('Monday','Moscow'),
                                 number_tracks('Wednesday','Moscow'),
                                 number_tracks('Friday','Moscow')],
                          ['СПБ',number_tracks('Monday','Saint-Petersburg'),
                                 number_tracks('Wednesday','Saint-Petersburg'),
                                 number_tracks('Friday','Saint-Petersburg')]], 
                    columns=['city', 'monday', 'wednesday', 'friday'])


,city,monday,wednesday,friday
0,МСК,15740,11056,15945
1,СПБ,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели
<a id='3.2'></a>
[Назад к содержанию](#section_id)

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [14]:
# Задаю таблицы с фильтром по городам
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [15]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day] # фильтр строк, у которых день равен day
    genre_df = genre_df[genre_df['time'] < time2]# фильтр строк, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] > time1]# фильтр строк, у которых время больше time1
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()# группировка по жанру и подсчет для каждого.
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)# сортировка по убыванию
    return genre_df_sorted[:10] # вывод топ-10 жанров

In [16]:
print('Топ-10 жанров в МСК в ПН утром')
genre_weekday(moscow_general, 'Monday', '07:00','11:00').to_frame()

Топ-10 жанров в МСК в ПН утром


,genre
genre,
pop,781
dance,549
electronic,480
rock,474
hiphop,286
ruspop,186
world,181
rusrap,175
alternative,164


In [17]:
print('Топ-10 жанров в СПБ в ПН утром')
genre_weekday(spb_general, 'Monday', '07:00','11:00').to_frame()

Топ-10 жанров в СПБ в ПН утром


,genre
genre,
pop,218
dance,182
rock,162
electronic,147
hiphop,80
ruspop,64
alternative,58
rusrap,55
jazz,44


In [18]:
print('Топ-10 жанров в МСК в ПТ вечером')
genre_weekday(moscow_general, 'Friday', '17:00','23:00')

Топ-10 жанров в МСК в ПТ вечером


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [19]:
print('Топ-10 жанров в СПБ в ПТ вечером')
genre_weekday(spb_general, 'Friday', '17:00','23:00')

Топ-10 жанров в СПБ в ПТ вечером


genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3 Жанровые предпочтения в Москве и Петербурге
<a id='3.3'></a>
[Назад к содержанию](#section_id)

Гипотеза: Петербург — столица хип-хопа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [20]:
(moscow_general.groupby('genre')['genre']
               .count()
               .sort_values(ascending=False)
               .head(10)
               .to_frame())

,genre
genre,
pop,5892
dance,4435
rock,3965
electronic,3786
hiphop,2096
classical,1616
world,1432
alternative,1379
ruspop,1372


In [21]:
(spb_general.groupby('genre')['genre']
               .count()
               .sort_values(ascending=False)
               .head(10)
               .to_frame())

,genre
genre,
pop,2431
dance,1932
rock,1879
electronic,1736
hiphop,960
alternative,649
classical,646
rusrap,564
ruspop,538


Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, хип-хоп одинаково популярен в Москве и Петербурге. 


## 4.Вывод
<a id='4'></a>
[Назад к содержанию](#section_id)

Утверждения проверены, выводы следующие:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первое утверждение полностью подтвердилось.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, второе утверждение подтвердилось лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третье утверждение не подтвердилось. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.
